## Model evaluation:

In this notebook, we load and estimate the images. To estimate, we use trained models from the DivBlurring_training.ipynb file.

## Import packages:

In [ ]:
import numpy as np
import torch
from tifffile import imread
from glob import glob
from tifffile import imsave
from IPython.display import clear_output
import pytorch_lightning as pl

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from math import log10, sqrt
dtype = torch.float
device = torch.device("cuda:0") 
print(device)
from Network import network

## Load the data.

In [ ]:
i = 8 # the data which is not used to train the model.
noisy_input= imread("./data/data4Sai/"+'BluryNoisy_tubulins_'+str(i)+'_SOFImodel.tif')
signal= imread("./data/data4Sai/"+'tubulins_'+str(i)+'_SOFImodel.tif')


In [ ]:
def PSNR(original, compressed):
    """Code snippet to calculate the PSNR value for the 100 samples.
    """
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):
     return 100
    max_pixel = np.max(original)
    psnr = 20 * log10(max_pixel / sqrt(mse))
    #  print(psnr)
    return psnr 
 
def predict_mmse(vae, img, samples, device, returnSamples=False, tq=True): 
    '''
    Predicts MMSE estimate.
    Parameters

    '''
    img_height,img_width=img.shape[0],img.shape[1]
    imgT=torch.Tensor(img.copy())
    image_sample = imgT.view(1,1,img_height,img_width).to(device)
    vae.num_samples = samples
    all_samples = np.array(vae(image_sample,tqdm_bar=tq))
    samples_array = all_samples[:,0,0,:,:]
    if returnSamples:
        return np.mean(samples_array,axis=0), samples_array
    else:
        return np.mean(samples_array,axis=0)

In [ ]:
def estimate_models(VAELightning,basedir,model_name,noisy_input,signal,method):
    """THe method to estimate and calculate the mean of PSNR for 100 estimates.
    """
    name = glob(basedir+"/"+model_name+'_last.ckpt')[0]
    vae = VAELightning.load_from_checkpoint(checkpoint_path = name)
    if not torch.cuda.is_available():
        raise ValueError("GPU not found, predictions will run on CPU and can be somewhat slow!")
    else:
        vae.to(device)
    
    imgMMSE, samps = predict_mmse(vae, noisy_input[0], samples=100, device=device, returnSamples=True)
    for i in range(1):
        plt.figure(figsize = (20,4))
        plt.title(label=method)
        plt.axis('off')
        plt.imshow(imgMMSE)
        plt.clim(vmin = np.min(signal[0]), vmax =np.max(signal[0])*(0.4))
        plt.show()
    psnr= []
    for i in range(100):
        imgMMSE, samps = predict_mmse(vae, noisy_input[i], samples=100, device=device, returnSamples=True)
        psnr_i = PSNR(signal[i],imgMMSE)
        psnr.append(psnr_i)

    print("The mean value of PSNR 100 predictions for "+ method +":"+str(np.mean(psnr)))
    print("Min value of predicted img:"+str(np.min(imgMMSE)))
    print("Max value of predicted img:"+str(np.max(imgMMSE)))

## True Signal and Noisy data.

In [ ]:
#The true signal and noisy data.

plt.figure(figsize = (60,8))
plt.subplot(211)
plt.title(label='True signal img')
plt.axis('off')
plt.imshow(signal[i])
plt.clim(vmin = np.min(signal[0]), vmax =np.max(signal[0])*(0.4))
plt.subplot(212)
plt.title(label='Observed Noisy img')
plt.axis('off')
plt.imshow(noisy_input[i])
plt.clim(vmin = np.min(signal[0]), vmax =np.max(signal[0])*(0.4))
plt.show()


## Ploting functions

In [ ]:
noisy_input = noisy_input
signal= signal
method = ['DivBlurring','DivBlurring_l1','DivBlurring_l2',
                    'DivBlurring_PCReg_1e3','DivBlurring_PCReg_1e5',
                    'DivBlurring_PCReg_l1'] # Definced approches.
model_name = ['models_DivBlurring','models_DivBlurring_l1Regu_1e10','models_DivBlurring_l2Regu_1e10',
                    'models_DivBlurring_PCReg_1e3','models_DivBlurring_PCReg_1e5',
                    'models_DivBlurring_PC_l1X_Reg_1e3_1e10'] # a name used to identify the model.
basedir = model_name # the base directory in which our model will be saved, we prefer same directory as model name.

In [ ]:
for i in range(len(method)):
    estimate_models(network.VAELightning,basedir[i],model_name[i],noisy_input,signal,method[i])

## Loss values comparision.

In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import tensorflow as tf
import glob
import pandas as pd
from pandas import DataFrame
from matplotlib.pyplot import axes

In [ ]:
all_dfs = {} 
for i in model_name:
    ea = event_accumulator.EventAccumulator(i, size_guidance={event_accumulator.SCALARS: 0})
    ea.Reload()
    dframes = DataFrame()
    dframes_total = DataFrame()
    mnames = ['reconstruction_loss_epoch', 'kl_loss_epoch', 'training_loss_epoch']


    for n in mnames:
        dframes = pd.DataFrame(ea.Scalars(n), columns=["wall_time", "epoch", n.replace('val/', '')])      
        dframes = dframes.drop(columns=['epoch','wall_time'])
        dframes_total[n] = dframes[n]
        dframes = DataFrame()
    
    all_dfs[i] = dframes_total

In [ ]:
for j in mnames:
    for i in model_name:
        plt.plot(all_dfs[i][j], label = i)

    plt.axis('off')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.title(j+' Loss values')
    plt.legend()
    plt.show()